In [122]:
import numpy as np
import pandas as pd
from gurobipy import *

In [123]:
#Vogel Approximation Method (VAM) to determine basic feasible solution

def VogelApproxMethod(supply, demand, distribution_costs):
    supply1 = supply
    demand1 = demand
    distribution_costs = distribution_costs
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply1[i]
        d = demand1[j]
        v = min(s, d)
        supply1[i] -= v
        demand1[j] -= v
        bfs.append(((i, j), v))
        if supply1[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand1[j] == 0 and j < len(demand) - 1:
            j += 1
    cost = 0
    for item in bfs:
        cost = cost + distribution_costs[item[0][0]][item[0][1]]*item[1]
    return bfs

In [124]:
#Helper Functions to determine optimum solution using Multiplier Method

def FindUandV(bfs, costs):
    u = [None] * len(costs)
    v = [None] * len(costs[0])
    u[0] = 0
    bfs1 = bfs.copy()
    while len(bfs1) > 0:
        for index, bv in enumerate(bfs1):
            i, j = bv[0]
            if u[i] is None and v[j] is None: continue
                
            cost = costs[i][j]
            if u[i] is None:
                u[i] = cost - v[j]
            else: 
                v[j] = cost - u[i]
            bfs1.pop(index)
            break
            
    return u, v

def FindW(bfs, costs, u, v):
    w = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                w.append(((i, j), u[i] + v[j] - cost))
    
    return w

def ImproveCheck(w):
    for p, v in w:
        if v > 0: return True
    return False

def FindEVPosition(w):
    w1 = w.copy()
    w1.sort(key=lambda w: w[1])
    return w1[-1][0]

def FindNextNode(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row
    
def FindLoop(bv_positions, ev_position):
    def Inner(loop):
        if len(loop) > 3:
            can_be_closed = len(FindNextNode(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = FindNextNode(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = Inner(loop + [next_node])
            if new_loop: return new_loop
    
    return Inner([ev_position])

def PivotLoop(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

In [125]:
#Recursively trying to find the optimum solution and the cost associated with it using Multiplier Method.

def MatrixMethod(supply, demand, costs):
    costs = costs.reshape(len(supply), len(demand)).tolist()

    def GoInner(bfs):
        u, v = FindUandV(bfs, costs)
        w = FindW(bfs, costs, u, v)
        if ImproveCheck(w):
            ev_position = FindEVPosition(w)
            loop = FindLoop([p for p, v in bfs], ev_position)
            return GoInner(PivotLoop(bfs, loop))
        return bfs
    
    basic_variables = GoInner(VogelApproxMethod(supply, demand, costs))
    ans = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        ans[i][j] = int(v)
        
    def FindTotalCost(costs, bfs):
        total_cost = 0
        for i, row in enumerate(costs):
            for j, cost in enumerate(row):
                total_cost += cost * bfs[i][j]
        return total_cost     
    return ans, FindTotalCost(costs, ans)

In [126]:
#Functions that generates balanced supply and demand with distribution costs.

def GenerateDistributionCosts(n_supply, n_demand):
    import numpy as np
    return np.random.randint(1, 10, n_supply*n_demand)

def RandomArrayWithConstSum(m, n):
    arr = [0] * m;
    for i in range(n) :
        arr[np.random.randint(0, n) % m] += 1;
    return np.asarray(arr)

def GenerateSupplyAndDemand(n_supply, n_demand):
    supply = np.random.randint(100, 300, n_supply)
    demand = RandomArrayWithConstSum(n_demand, np.sum(supply))
    return (supply, demand)

In [139]:
#Function that calculates Total Supply Chain Distribution Cost 

def TotalBalancedSupplyChainDistributionCostEachProduct(product_qty, n_suppliers, n_plants, n_hubs, n_stockist, n_distributors):
    
    s4, d4 = GenerateSupplyAndDemand(n_stockist, n_distributors)
    t4 = GenerateDistributionCosts(n_stockist, n_distributors)
    
    s3, d3 = GenerateSupplyAndDemand(n_hubs, n_stockist)
    t3 = GenerateDistributionCosts(n_hubs, n_stockist)
    
    s2, d2 = GenerateSupplyAndDemand(n_plants, n_hubs)
    t2 = GenerateDistributionCosts(n_plants, n_hubs)
    
    s1, d1 = GenerateSupplyAndDemand(n_suppliers, n_plants)
    t1 = GenerateDistributionCosts(n_suppliers, n_plants)
    
    bfs4, cost4 = MatrixMethod(s4, d4, t4)
    bfs3, cost3 = MatrixMethod(s3, d3, t3)
    bfs2, cost2 = MatrixMethod(s2, d2, t2)
    bfs1, cost1 = MatrixMethod(s1, d1, t1)
    
    return product_qty*(cost4 + cost3 + cost2 + cost1)

In [128]:
#Vehicle Scheduling Problem

In [129]:
def MultiProduct(n_products):
    product_quantity = []
    for i in range(n_products):
        product_quantity.append(np.random.randint(1, 15))
    return product_quantity

In [130]:
def MultiModeCostWeightsAndAvailability(n_modes):
    capacity = []
    mode_transport_costs = []
    availability = []
    for i in range (n_modes):
        n_vehicles_each_mode = np.random.randint(1, 3)
        cost_each_mode = [] 
        availability_each_mode = []
        capacity_each_mode = []
        for j in range(n_vehicles_each_mode):
            cost_each_mode.append(np.random.randint(1, 10))
            capacity_each_mode.append(np.random.randint(1, 5))
            availability_each_mode.append(np.random.randint(5, 10))
        mode_transport_costs.append(cost_each_mode)
        capacity.append(capacity_each_mode)
        availability.append(availability_each_mode)
    return mode_transport_costs, capacity, availability

In [131]:
def FindOptimumTranportCostPerLayer(mode_transport_costs, capacity, availability, weight_transported):
    OptimizeModel = Model("OptimizeModel")
    OptimizeModel.setParam('OutputFlag', False )
    count = 0
    for i in range(len(mode_transport_costs)):
        count = count + len(mode_transport_costs[i])

    xs = []
    for i in range (count):
        xs.append(OptimizeModel.addVar(lb = 0, vtype = GRB.INTEGER, name = "x" + str(i)))

    ys = []
    for i in range(len(mode_transport_costs)):
        for j in range(len(mode_transport_costs[i])):
            ys.append(mode_transport_costs[i][j])

    cost = 0
    for i in range(len(ys)):
        cost = cost + ys[i]*xs[i]

    caps = []
    for i in range(len(capacity)):
        for j in range(len(capacity[i])):
            caps.append(capacity[i][j])

    weight = 0
    for i in range(len(caps)):
        weight = weight + caps[i]*xs[i]


    avails = []
    for i in range(len(availability)):
        for j in range(len(availability[i])):
            avails.append(availability[i][j])


    for i in range(len(avails)):
        OptimizeModel.addConstr(xs[i] <= avails[i])

    OptimizeModel.addConstr(weight_transported <= weight)
    OptimizeModel.setObjective(cost, GRB.MINIMIZE)
    OptimizeModel.optimize()
    
    x_vals = []
    for var in OptimizeModel.getVars():
        x_vals.append([var.varName, ' = ', var.x])
    obj_cost = OptimizeModel.objVal
    return x_vals, obj_cost

In [179]:
#Obtaining Results

In [180]:
results = []

In [181]:
for i in range(10):
    n_products = 2
    n_stockist = np.random.randint(1, 10)
    n_suppliers = np.random.randint(1, 10)
    n_plants = np.random.randint(1, 10)
    n_hubs = np.random.randint(1, 10)
    n_distributors = np.random.randint(1, 10)
    product_qty = MultiProduct(n_products)
    total_cost = 0
    vs_cost = 0
    total_weight = 0
    total_vs_cost = 0
    for k in range(4):
        mode_transport_costs, capacity, availability = MultiModeCostWeightsAndAvailability(3)
        weight_transported = np.random.randint(1,3)*(sum(product_qty))
        total_weight = total_weight + weight_transported
        x_vals, vs_cost = FindOptimumTranportCostPerLayer(mode_transport_costs, capacity, availability, weight_transported)
        total_vs_cost = total_vs_cost + vs_cost
    for j in product_qty:
        total_cost = total_cost + TotalBalancedSupplyChainDistributionCostEachProduct(j, n_suppliers, n_plants, n_hubs, n_stockist, n_distributors)
    
    results.append([n_suppliers, n_plants, n_hubs, n_stockist, n_distributors, n_products, product_qty, total_cost, 
                    total_vs_cost, (total_cost + total_vs_cost)])

In [182]:
results = pd.DataFrame(results)
results.columns = ('Suppliers', 'Plants', 'Hubs', 'Stockists', 'Distributors', 'Products', 'Product Quantity', 'Distribution Cost', 
                   'VS Cost','Grand Total')

In [183]:
results

,Suppliers,Plants,Hubs,Stockists,Distributors,Products,Product Quantity,Distribution Cost,VS Cost,Grand Total
0,7,2,2,7,8,2,"[4, 12]",195612.0,8320.0,203932.0
1,1,6,1,4,1,2,"[2, 8]",122776.0,3990.0,126766.0
2,6,9,9,7,5,2,"[2, 4]",85404.0,900.0,86304.0
3,8,2,7,8,6,2,"[11, 7]",269675.0,12564.0,282239.0
4,6,6,6,8,9,2,"[7, 13]",248507.0,2160.0,250667.0
5,7,9,6,7,1,2,"[12, 6]",278586.0,7650.0,286236.0
6,1,1,8,4,1,2,"[5, 5]",102155.0,1380.0,103535.0
7,9,9,2,5,5,2,"[6, 8]",235694.0,8568.0,244262.0
8,3,8,4,3,5,2,"[4, 14]",163330.0,6552.0,169882.0
9,6,4,9,2,7,2,"[8, 13]",267591.0,15960.0,283551.0
